In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np

In [2]:
"""
master_input.parquet
master_output.parquet
play_level.parquet
player_analysis.parquet
spatial_features.parquet
supplementary.parquet
trajectories.parquet
"""
base_dir = "../consolidated_data/"

In [118]:
game_id = '2023090700'
play_id = '1679'
nfl_id = '46243'
play_keys = ['game_id', 'play_id']
player_keys = play_keys + ['nfl_id']
player_dtls = ['player_name', 'player_height', 'player_weight', 'player_birth_date', 'player_position', 'player_side', 'player_role', 'num_frames_output']
player_mvmt = ['frame_id', 'x', 'y']

input_cols = player_keys + player_dtls + player_mvmt + ['player_to_predict', 'play_direction', 'absolute_yardline_number', 's', 'a', 'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'week']
input_df = pd.read_parquet(f'{base_dir}/master_input.parquet')

output_cols = player_keys + player_mvmt
output_df = pd.read_parquet(f'{base_dir}/master_output.parquet')

supp_df = pd.read_parquet(f'{base_dir}/supplementary.parquet')

In [104]:
supp_df.columns

Index(['game_id', 'season', 'week', 'game_date', 'game_time_eastern',
       'home_team_abbr', 'visitor_team_abbr', 'play_id', 'play_description',
       'quarter', 'game_clock', 'down', 'yards_to_go', 'possession_team',
       'defensive_team', 'yardline_side', 'yardline_number',
       'pre_snap_home_score', 'pre_snap_visitor_score',
       'play_nullified_by_penalty', 'pass_result', 'pass_length',
       'offense_formation', 'receiver_alignment', 'route_of_targeted_receiver',
       'play_action', 'dropback_type', 'dropback_distance',
       'pass_location_type', 'defenders_in_the_box', 'team_coverage_man_zone',
       'team_coverage_type', 'penalty_yards', 'pre_penalty_yards_gained',
       'yards_gained', 'expected_points', 'expected_points_added',
       'pre_snap_home_team_win_probability',
       'pre_snap_visitor_team_win_probability',
       'home_team_win_probability_added', 'visitor_team_win_probility_added'],
      dtype='object')

In [52]:
pd.merge(input_df, supp_df, on=['game_id', 'play_id'], how='left')

game_id                                  4880579
play_id                                  4880579
player_to_predict                        4880579
nfl_id                                   4880579
frame_id                                 4880579
                                          ...   
expected_points_added                    4880579
pre_snap_home_team_win_probability       4880579
pre_snap_visitor_team_win_probability    4880579
home_team_win_probability_added          4880579
visitor_team_win_probility_added         4880579
Length: 63, dtype: int64

In [5]:
output_xdf = pd.merge(
    output_df[output_cols], 
    input_df[player_keys + player_dtls].drop_duplicates(), 
    on=player_keys, 
    how='left'
)

In [67]:
hour = supp_df['game_clock'].iloc[0].split(':')[0]

In [136]:
season = 2023
team = 'KC'

# build predicate builder based on criteria for easy searching
games_df = supp_df.query(f"season == {season} and possession_team == '{team}'").sort_values('game_date')
games_df[['game_id', 'home_team_abbr', 'visitor_team_abbr']].drop_duplicates()

,game_id,home_team_abbr,visitor_team_abbr
13702,2024010709,LAC,KC
48,2023090700,KC,DET
1180,2023091705,JAX,KC
2389,2023092411,KC,CHI
3307,2023100113,NYJ,KC
4000,2023100810,MIN,KC
4133,2023101200,KC,DEN
5461,2023102209,KC,LAC
6255,2023102911,DEN,KC
6466,2023110500,KC,MIA


In [138]:
game_id = 2023102209
quarter = 2
minute = 14
possession_team = 'KC'
cols = play_keys + ['game_clock', 'pass_result', 'yards_gained']

game_df = games_df.query(f"game_id == {game_id} and quarter == {quarter}").loc[supp_df['game_clock'].str.split(':').str[0] == f'{str(minute).zfill(2)}'][cols]
game_df
# game_df.query(f"possession_team == '{possession_team}'")[['play_description', 'pass_result', 'game_clock']].sort_values('game_clock')

,game_id,play_id,game_clock,pass_result,yards_gained
5420,2023102209,937,14:23,C,46
